In [ ]:
HF_TOKEN = input('토큰을 입력하세요')

## 라이브러리

In [17]:
import transformers
import torch
from transformers import AutoTokenizer
import os
import json
import time

## 프롬프트

In [49]:
system_prompt = """You are a kind spelling checker.
    When sentences written in various languages are entered, please correct them according to the grammar of the entered language.
    Please print out both the input and the revised sentences, but also tell me what content is wrong and how it was corrected.
    Please let me know how many places have been modified in numbers.
    Please provide information about what language you entered is and attach the translation into Korean.
    However, if the language entered is Korean, please do not print out the translated sentence.
    If you do not find spelling errors in the entered sentences, please print them out together that you did not find spelling errors instead of corrected sentences.
    If you don't know the grammar of the language of the input sentence, please print it out as an unknown language instead of a modified sentence.
    If the user asks about spelling sentences asked in the previously presented language while presenting the language, please print out the revised sentences and the sentences translated into Korean.
    However, if you have never entered a sentence in the language presented by the user before, please let me know that it is a language you have never entered before.
    Please print out the answer format concisely and neatly."""

#### Llama3

In [8]:
llama = 'meta-llama/Llama-3.2-1B-Instruct'

pipeline = transformers.pipeline(
    'text-generation',
    model=llama,
    model_kwargs={'torch_dtype': torch.bfloat16},
    device_map='auto',
    token=HF_TOKEN
)

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [53]:
sample_messages = [
    {'role': 'system',
     'content': 'You are a helpful AI assistant who answers question in a friendly and informal manner.'},
    {'role': 'user',
     'content': 'Tell us about Transformers self-attention.'}
]

tokenizer = AutoTokenizer.from_pretrained(llama)
chat_llama = tokenizer.apply_chat_template(sample_messages, add_generation_prompt=True)
tokenizer.decode(chat_llama)

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 27 Nov 2024\n\nYou are a helpful AI assistant who answers question in a friendly and informal manner.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nTell us about Transformers self-attention.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n'

In [54]:
llama_path = '/content/llama_history_en.json'
terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids('<|eot_id|>')
]

def llama_load_history():
    if os.path.exists(llama_path):
        with open(llama_path, 'r', encoding='utf-8') as file:
            return json.load(file)
    return [{'role': 'system', 'content': system_prompt}]

def llama_save_history(messages):
    with open(llama_path, 'w', encoding='utf-8') as file:
        json.dump(messages, file, ensure_ascii=False, indent=4)

def llama_get(messages):
    chat_temp = tokenizer.apply_chat_template(messages, add_generation_prompt=True)
    response = pipeline(
        messages,
        eos_token_id=terminators,
        max_new_tokens=512,
        do_sample=False
    )
    return response[0]['generated_text'][-1]['content']

In [56]:
llama_messages = llama_load_history()
print('안녕하세요! 맞춤법 검사를 도와드리겠습니다.\n')

while True:
    llama_input = input('[사용자]: ')
    print(f'[사용자]: {llama_input}\n')
    if llama_input.lower() in ['exit', 'quit', '종료', '나가기']:
        print('대화를 종료합니다.\n')
        llama_save_history(llama_messages)
        break
    llama_messages.append({'role': 'user', 'content': llama_input})
    start_time = time.time()
    llama_response = llama_get(llama_messages)
    llama_messages.append({'role': 'assistant', 'content': llama_response})
    print(f'[챗봇]: {llama_response}\n')
    print(f'토큰 수: {len}')
    end_time = time.time()
    print(f'답변 생성 시간: {end_time-start_time:.2f}초\n')

안녕하세요! 맞춤법 검사를 도와드리겠습니다.

[사용자]: 이렇거 하고 십지 안고, 해서도 않되는 일이야.


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[사용자]: 이렇거 하고 십지 안고, 해서도 않되는 일이야.

[챗봇]: **Input:** 
이렇거고 십지 안고, 해서도 않되는 일이야.

**Revised Sentence:** 
이렇고 십지 안고, 해서도 안되는 일이야.

**Content Wrong:**
- "이렇거고"는 "이렇고"와 "이렇다"를 의미하는 문장의 시작을 나타내는 문장의 시작을 나타내는 문장입니다. 
- "십지 안고"는 "십지"와 "안고"를 의미하는 문장의 시작을 나타내는 문장의 시작을 나타내는 문장입니다. 
- "해서도"는 "해서"와 "도"를 의미하는 문장의 시작을 나타내는 문장의 시작을 나타내는 문장입니다. 
- "안되는 일이야"는 "안되는"과 "일이야"를 의미하는 문장의 시작을 나타내는 문장입니다.

**Number of Modifications:** 
- 1. "이렇거고" -> "이렇고"
- 2. "십지 안고" -> "십지 안고"
- 3. "해서도" -> "해서도"
- 4. "안되는 일이야" -> "안되는 일이야"

**Language:** 
이 언어는 한국어입니다.

**Unknown Language:** 
이 언어는 한국어입니다.

**Spelling Errors:** 
- None found.

**Revised Sentences:** 
이렇고 십지 안고, 해서도 안되는 일이야.

토큰 수: <built-in function len>
답변 생성 시간: 9.25초

[사용자]: Hier, je vais à l'école avec mon amis. Nous avons etudier beaucoup.


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[사용자]: Hier, je vais à l'école avec mon amis. Nous avons etudier beaucoup.

[챗봇]: **Input:** 
Hier, je vais à l'école avec mon amis. Nous avons etudier beaucoup.

**Revised Sentence:** 
Hier, je vais à l'école avec mes amis. Nous avons étudié beaucoup.

**Content Wrong:**
- "je vais"는 "je vais"와 " vais"를 의미하는 문장의 시작을 나타내는 문장의 시작을 나타내는 문장입니다. 
- "mon amis"는 "mon amis"와 " amis"를 의미하는 문장의 시작을 나타내는 문장입니다. 
- "Nous avons etudier"는 "Nous avons"와 "etudier"를 의미하는 문장의 시작을 나타내는 문장입니다. 
- "etudier"는 "etudier"와 "etudier"를 의미하는 문장의 시작을 나타내는 문장입니다. 
- " beaucoup"는 " beaucoup"와 " beaucoup"를 의미하는 문장의 시작을 나타내는 문장입니다.

**Number of Modifications:** 
- 1. "je vais" -> "je vais"
- 2. "mon amis" -> "mes amis"
- 3. "Nous avons" -> "Nous avons"
- 4. "etudier" -> "étudié"
- 5. " beaucoup" -> " beaucoup"

**Language:** 
이 언어는 한국어입니다.

**Unknown Language:** 
이 언어는 한국어입니다.

**Spelling Errors:** 
- None found.

**Revised Sentences:** 
Hier, je vais à l'école avec mes amis. Nous avons étudié beaucoup.

**Translated

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[사용자]: Ayer, yo y mi amigo fueron a la tienda. Compramos café y comimos lo.

[챗봇]: **Input:** 
Ayer, yo y mi amigo fueron a la tienda. Compramos café y comimos lo.

**Revised Sentence:** 
Ayer, yo y mi amigo fueron a la tienda. Compramos café y comimos lo.

**Content Wrong:**
- "Ayer"는 "Ayer"와 "ayer"를 의미하는 문장의 시작을 나타내는 문장의 시작을 나타내는 문장입니다. 
- "yo y mi amigo"는 "yo y"를 의미하는 문장의 시작을 나타내는 문장입니다. 
- "fueron a la tienda"는 "fueron"와 "a la tienda"를 의미하는 문장의 시작을 나타내는 문장입니다. 
- "Compramos café y comimos lo"는 "Compramos"와 "y comimos"를 의미하는 문장의 시작을 나타내는 문장입니다. 
- "lo"는 "lo"와 "lo"를 의미하는 문장의 시작을 나타내는 문장입니다.

**Number of Modifications:** 
- 1. "Ayer" -> "Ayer"
- 2. "yo y mi amigo" -> "yo y mi amigo"
- 3. "fueron a la tienda" -> "fueron a la tienda"
- 4. "Compramos café y comimos lo" -> "Compramos café y comimos lo"

**Language:** 
이 언어는 한국어입니다.

**Unknown Language:** 
이 언어는 한국어입니다.

**Spelling Errors:** 
- None found.

**Revised Sentences:** 
Ayer, yo y mi amigo fueron a la tienda. Compramos café y co

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[사용자]: I går, min vän och jag gick till butiken. Vi köpte kaffe och drack det.

[챗봇]: **Input:** 
I går, min vän och jag gick till butiken. Vi köpte kaffe och drack det.

**Revised Sentence:** 
I går, min vän och jag gick till butiken. Vi köpte kaffe och drack det.

**Content Wrong:**
- "I går"는 "I går"와 "i går"를 의미하는 문장의 시작을 나타내는 문장입니다. 
- "min vän"는 "min vän"와 "vän"를 의미하는 문장의 시작을 나타내는 문장입니다. 
- "och"는 "och"와 "och"를 의미하는 문장의 시작을 나타내는 문장입니다. 
- "jag gick"는 "jag gick"와 "gick"를 의미하는 문장의 시작을 나타내는 문장입니다. 
- "till butiken"는 "till"와 "butiken"를 의미하는 문장의 시작을 나타내는 문장입니다. 
- "Vi köpte kaffe och drack det"는 "Vi köpte"와 "och drack"를 의미하는 문장의 시작을 나타내는 문장입니다. 
- "det"는 "det"와 "det"를 의미하는 문장의 시작을 나타내는 문장입니다.

**Number of Modifications:** 
- 1. "I går" -> "I går"
- 2. "min vän" -> "min vän"
- 3. "och" -> "och"
- 4. "jag gick" -> "jag gick"
- 5. "till butiken" -> "till butiken"
- 6. "Vi köpte" -> "Vi köpte"
- 7. "och drack" -> "och drack"

**Language:** 
이 언어는 한국어입니다.

**Unknown Language:** 
이 언어는 한국어입

In [57]:
llama_messages = llama_load_history()
print('안녕하세요! 맞춤법 검사를 도와드리겠습니다.\n')

while True:
    llama_input = input('[사용자]: ')
    print(f'[사용자]: {llama_input}\n')
    if llama_input.lower() in ['exit', 'quit', '종료', '나가기']:
        print('대화를 종료합니다.\n')
        llama_save_history(llama_messages)
        break
    llama_messages.append({'role': 'user', 'content': llama_input})
    start_time = time.time()
    llama_response = llama_get(llama_messages)
    llama_messages.append({'role': 'assistant', 'content': llama_response})
    print(f'[챗봇]: {llama_response}\n')
    end_time = time.time()
    print(f'답변 생성 시간: {end_time-start_time:.2f}초\n')

안녕하세요! 맞춤법 검사를 도와드리겠습니다.

[사용자]: Can you tell me the Korean sentence you asked earlier?


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[사용자]: Can you tell me the Korean sentence you asked earlier?

[챗봇]: 이 언어는 한국어입니다.

이 언어는 한국어입니다.

답변 생성 시간: 2.60초

[사용자]: Can you tell me the Korean sentence I asked earlier?


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[사용자]: Can you tell me the Korean sentence I asked earlier?

[챗봇]: 이 언어는 한국어입니다.

이 언어는 한국어입니다.

답변 생성 시간: 2.63초

[사용자]: 종료
[사용자]: 종료

대화를 종료합니다.

